# 연도별 물동량 / 체류시간

In [301]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats

In [302]:
# 연간 물동량
busan_ton_raw=pd.read_excel('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/busan_ton.xlsx')
busan_ton=busan_ton_raw.rename(columns={'총계':'물동량'})
busan_ton

,년도,물동량
0,2013,324858224
1,2014,346640071
2,2015,359676343
3,2016,362369364
4,2017,401232669
5,2018,461461501
6,2019,461122794
7,2020,410522629
8,2021,442558714
9,2022,424924985


In [303]:
# 체류 시간
stayed_time_raw=pd.read_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/shipSchedule(12 ~ 23).csv')
# 접안시간 기준으로 년도 행과 월 행 생성
year=[]
for i in range(len(stayed_time_raw)):
    year.append(int(stayed_time_raw['접안시간'][i][:4]))
stayed_time_raw['년도']=year
year_stayed_time=stayed_time_raw.groupby('년도',as_index=False)['체류시간_totalTime(시간)'].sum()
year_stayed_time=year_stayed_time.drop([0,1]).reset_index().drop('index',axis=1)
year_stayed_time

,년도,체류시간_totalTime(시간)
0,2013,13350
1,2014,13234
2,2015,17719
3,2016,17966
4,2017,17321
5,2018,34056
6,2019,65172
7,2020,93754
8,2021,126737
9,2022,194814


In [304]:
ton_stayed_df=busan_ton.merge(year_stayed_time,on='년도')
ton_stayed_df.to_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/물동량-체류시간.csv')

In [305]:
ton_stayed_df.rename(columns={'체류시간_totalTime(시간)':'체류시간'},inplace=True)
ton_stayed_df.set_index('년도',inplace=True)
scaler=MinMaxScaler()
ton_stayed_scaled=pd.DataFrame(
    scaler.fit_transform(ton_stayed_df.copy()),columns=ton_stayed_df.columns
    )
ton_stayed_scaled.index=ton_stayed_df.index
ton_stayed_scaled.reset_index(inplace=True)
ton_stayed_scaled

,년도,물동량,체류시간
0,2013,0.000000,0.000617
1,2014,0.159453,0.000000
2,2015,0.254885,0.023871
3,2016,0.274599,0.025185
4,2017,0.559097,0.021752
5,2018,1.000000,0.110822
6,2019,0.997521,0.276432
7,2020,0.627104,0.428555
8,2021,0.861623,0.604102
9,2022,0.732536,0.966432


### 1. 시계열 그래프

In [306]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=ton_stayed_scaled['년도'],y=ton_stayed_scaled['물동량'],name='물동량'))
fig.add_trace(go.Scatter(x=ton_stayed_scaled['년도'],y=ton_stayed_scaled['체류시간'],name='체류시간'))
fig.update_layout(
    title='연간 물동량, 체류시간 추이',
    xaxis=dict(title='년도'),
    annotations=[
        dict(x=-0.03,
             y=1.15,
             xref='paper',
             yref='paper',
             text='(모든 수치는 정규화된 수치임)',
             showarrow=False)
    ]
)

### 2.상관분석

In [307]:
corr=stats.pearsonr(ton_stayed_scaled['물동량'],ton_stayed_scaled['체류시간'])
corr

PearsonRResult(statistic=0.5513365281600163, pvalue=0.07873540099886712)

#### 상관계수 : 0.551, pvalue=0.079

In [308]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=ton_stayed_scaled['물동량'],y=ton_stayed_scaled['체류시간'],
    name='실측값',mode='markers'
))
fig.add_trace(go.Scatter(
    x=ton_stayed_scaled['물동량'],
    y=np.poly1d(np.polyfit(ton_stayed_scaled['물동량'],ton_stayed_scaled['체류시간'],1))\
        (ton_stayed_scaled['물동량']),
    mode='lines',line=dict(color='red'),name='추세선'
))
fig.update_layout(title='연간 물동량 - 체류시간',
                  xaxis=dict(title='물동량'),
                  yaxis=dict(title='체류시간'),
                  annotations=[
                      dict(x=-0.02,
                           y=1.15,
                           xref='paper',
                           yref='paper',
                           text=f'상관계수 : {round(corr[0],3)}',
                           showarrow=False)
                  ])

# 연도별 물동량 / 화물처리 실적(Done)

In [309]:
# 물동량
busan_ton

,년도,물동량
0,2013,324858224
1,2014,346640071
2,2015,359676343
3,2016,362369364
4,2017,401232669
5,2018,461461501
6,2019,461122794
7,2020,410522629
8,2021,442558714
9,2022,424924985


In [310]:
# 화물 처리 실적
shipment_df_raw=pd.read_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/년,월별 물동량.csv')
shipment_df_raw.drop(['Unnamed: 0','년/월'],axis=1,inplace=True)  #불필요한 행 제거
shipment_df=shipment_df_raw.groupby('년도',as_index=False)['컨테이너'].sum()
shipment_df.rename(columns={'컨테이너':'화물처리실적'},inplace=True)
shipment_df

,년도,화물처리실적
0,2013,299659802
1,2014,321495703
2,2015,333051558
3,2016,336585345
4,2017,375862809
5,2018,439631441
6,2019,447341103
7,2020,390306842
8,2021,420019212
9,2022,401331613


In [311]:
# merge
ton_shipment_df=busan_ton.merge(shipment_df,on='년도')
ton_shipment_df.to_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/물동량-화물처리실적.csv')

### 1.상관분석

In [312]:
ton_shipment_df.set_index('년도',inplace=True)

In [313]:
ton_shipment_scaled=pd.DataFrame(
    scaler.fit_transform(ton_shipment_df),columns=ton_shipment_df.columns
)
ton_shipment_scaled.index=ton_shipment_df.index
ton_shipment_scaled.reset_index(inplace=True)
ton_shipment_scaled

,년도,물동량,화물처리실적
0,2013,0.000000,0.000000
1,2014,0.159453,0.147858
2,2015,0.254885,0.226107
3,2016,0.274599,0.250035
4,2017,0.559097,0.515996
5,2018,1.000000,0.947795
6,2019,0.997521,1.000000
7,2020,0.627104,0.613802
8,2021,0.861623,0.814994
9,2022,0.732536,0.688454


In [314]:
corr2=stats.pearsonr(ton_shipment_scaled['물동량'],ton_shipment_scaled['화물처리실적'])
corr2

PearsonRResult(statistic=0.9987150406084448, pvalue=5.709756835309294e-13)

In [315]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=ton_shipment_scaled['년도'],y=ton_shipment_scaled['물동량'],
                         mode='lines',name='물동량'))
fig.add_trace(go.Scatter(x=ton_shipment_scaled['년도'],y=ton_shipment_scaled['화물처리실적'],
                         mode='lines',name='화물처리실적'))
fig.update_layout(title='연간 물동량, 화물처리실적 추이',
                  xaxis=dict(title='년도'),
                  annotations=[
                      dict(x=-0.03,
                           y=1.15,
                           xref='paper',
                           yref='paper',
                           text='(모든 수치는 정규화된 수치임)',
                           showarrow=False)])

In [316]:
fig=go.Figure()
fig.add_trace(go.Scatter(
    x=ton_shipment_scaled['물동량'],
    y=ton_shipment_scaled['화물처리실적'],
    mode='markers',name='실측값'
))
fig.add_trace(go.Scatter(
    x=ton_shipment_scaled['물동량'],
    y=np.poly1d(np.polyfit(ton_shipment_scaled['물동량'],ton_shipment_scaled['화물처리실적'],1))\
        (ton_shipment_scaled['물동량']),
    mode='lines',name='추세선'
))
fig.update_layout(title='연간 물동량 - 화물처리실적 상관분석',
                  xaxis=dict(title='물동량'),
                  yaxis=dict(title='화물처리실적'),
                  annotations=[
                      dict(x=-0.03,
                           y=1.15,
                           xref='paper',
                           yref='paper',
                           text=f'상관계수 : {round(corr2[0],3)}\t(정규화된 수치로 계산)',
                           showarrow=False)])

# 연도별 물동량 / 출입항 외국인 수

In [317]:
# 출입항 외국인 선원 수 dataframe 구축
people_num_raw=pd.read_excel('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data//항구별 출입항 선원_1.xlsx',
              header=[0,1,2],index_col=0)

year_list=list(range(2009,2022))
year_list
col_list=[]
for i in year_list:
    col=people_num_raw[str(i)][('입항','선원수 (명)')][1]
    col_list.append(col)

busan_people_num_df=pd.DataFrame(col_list,columns=['입항 선원수(명)'],index=year_list)
busan_people_num_df=busan_people_num_df.reset_index().rename(columns={'index':'년도'})
# merge
ton_people_df=busan_ton.merge(busan_people_num_df,on='년도')
ton_people_df.to_csv('C:/playdata projects/Adios/DA34-1st-Adios-BusanPort/data/물동량-입항 선원 수.csv')

c:\playdata projects\Adios\DA34-1st-Adios-BusanPort\.conda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default

C:\Users\kamci\AppData\Local\Temp\ipykernel_10260\2009752023.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [318]:
busan_people_num_df

,년도,입항 선원수(명)
0,2009,418044
1,2010,477094
2,2011,476000
3,2012,517025
4,2013,542934
5,2014,534198
6,2015,522605
7,2016,686959
8,2017,554548
9,2018,529778


In [319]:
# 연도별 입항 외국인 선원 수 추이
fig=go.Figure()
fig.add_trace(go.Scatter(x=busan_people_num_df['년도'],y=busan_people_num_df['입항 선원수(명)'],
                         mode='markers+lines',name='실측값'))
fig.add_trace(go.Scatter(x=busan_people_num_df['년도'],
                         y=np.poly1d(np.polyfit(busan_people_num_df['년도'],busan_people_num_df['입항 선원수(명)'],1))\
                            (busan_people_num_df['년도']),
                        mode='lines',line=dict(color='red'),
                        name='추세선'))
fig.update_layout(title='연간 입항 외국인 선원수 추이',
                  xaxis=dict(title='년도'),
                  yaxis=dict(title='입항 선원 수(단위:명)'))

In [320]:
# 연도별 입항 외국인 선원 수 추이 - 바그래프
fig=go.Figure()
fig.add_trace(go.Bar(x=busan_people_num_df['년도'],y=busan_people_num_df['입항 선원수(명)'],
                     text=busan_people_num_df['입항 선원수(명)'],textposition='outside'))
fig.update_layout(title='연간 외국인 선원 수 ',
                  xaxis=dict(title='년도'),
                  yaxis=dict(title='연간 외국인 선원 수(단위:명)'))

In [321]:
ton_people_df

,년도,물동량,입항 선원수(명)
0,2013,324858224,542934
1,2014,346640071,534198
2,2015,359676343,522605
3,2016,362369364,686959
4,2017,401232669,554548
5,2018,461461501,529778
6,2019,461122794,542861
7,2020,410522629,419447
8,2021,442558714,378308


### 1. 상관분석

In [323]:
ton_people_df

,년도,물동량,입항 선원수(명)
0,2013,324858224,542934
1,2014,346640071,534198
2,2015,359676343,522605
3,2016,362369364,686959
4,2017,401232669,554548
5,2018,461461501,529778
6,2019,461122794,542861
7,2020,410522629,419447
8,2021,442558714,378308


In [322]:
ton_people_scaled=pd.DataFrame(scaler.fit_transform(ton_people_df.set_index('년도')),
                               columns=ton_people_df.columns[1:],index=year_list)
ton_people_scaled=ton_people_scaled.reset_index().rename(columns={'index':'년도'})
ton_people_scaled

ValueError: Shape of passed values is (9, 2), indices imply (13, 2)

In [ ]:
corr3=stats.pearsonr(ton_people_scaled['물동량'],ton_people_scaled['입항 선원수(명)'])
corr3

PearsonRResult(statistic=-0.38866509147294365, pvalue=0.30123874419770674)

### 히트맵
* x : 화물처리 실적
* y : 물동량
* 색 : 체류 시간

# 화물처리 실적 - 체류시간

In [ ]:
shipment_df

,년도,화물처리실적
0,2013,299659802
1,2014,321495703
2,2015,333051558
3,2016,336585345
4,2017,375862809
5,2018,439631441
6,2019,447341103
7,2020,390306842
8,2021,420019212
9,2022,401331613
